In [1]:
%matplotlib inline
import sys, os
sys.path.append('../../')
import h5py
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from decimal import Decimal
#from tools import OHLC, Density
#from tools import open_file, products
from tools.instruments import instruments
from tools.display import highchart
from tools.datatools import norm
from IPython.display import clear_output
import time

In [2]:
from sktime.base import load
from sktime.classification.deep_learning.cnn import CNNClassifier

* 모델 로드

In [3]:
period  = 20
modelfile = os.path.join('..','..','data','trend index','models',f'trend_{period}day')
model = load(modelfile)

In [4]:
savefilepath = os.path.join('..','..','data','trend index','trend_index.hdf')
#savefile = h5py.File(savefilepath, 'a')

In [5]:
quotes = instruments.quotes(fields='ohlc')

In [6]:
symbols = list(quotes.columns.levels[0])
for symbol in symbols:
    savefile = h5py.File(savefilepath, 'a')
    print(symbol)
    quote = quotes[symbol].dropna()
    data=[]
    dates = []
    for group in quote.rolling(period):
        if len(group) != period:
            continue
        
        dates.append(group.index.values.max())
        data.append(group.values)
        
    dates = np.array(dates).astype('M8[s]').astype('int64').reshape((len(dates), 1))
    data = np.array(list(map(norm, data)))
    data = np.moveaxis(data, 1, 2)
    
    print("확률 생성중")
    trend = model.predict_proba(data)
    arr = np.concatenate((dates,trend), axis=1)
    
    if symbol not in savefile.keys():
        savefile.create_group(symbol)
    
    savefile[symbol].create_dataset(f"trend{period}", arr.shape, data=arr)
    savefile.close()
    

AD
확률 생성중
2225/2225 [==============================] - 1s 597us/step
BO
확률 생성중
3869/3869 [==============================] - 2s 613us/step
BP
확률 생성중
2953/2953 [==============================] - 2s 596us/step
C
확률 생성중
3923/3923 [==============================] - 2s 585us/step
CC
확률 생성중
3277/3277 [==============================] - 2s 595us/step
CD
확률 생성중
2842/2842 [==============================] - 2s 588us/step
CL
확률 생성중
2453/2453 [==============================] - 1s 588us/step
CT
확률 생성중
3159/3159 [==============================] - 2s 586us/step
DX
확률 생성중
1500/1500 [==============================] - 1s 597us/step
EC
확률 생성중
1469/1469 [==============================] - 1s 596us/step
ES
확률 생성중
1569/1569 [==============================] - 1s 638us/step
FV
확률 생성중
2132/2132 [==============================] - 1s 597us/step
GC
확률 생성중
2972/2972 [==============================] - 2s 584us/step
HG
확률 생성중
3929/3929 [==============================] - 2s 583us/step
HO
확률 생성중
2666/2666 [==============

### Verification

In [3]:
savefilepath = os.path.join('..','..','data','trend index','trend_index.hdf')

In [4]:
file=h5py.File(savefilepath, mode='r')

In [26]:
symbol = 'GC'
quote = instruments[symbol].quotes(fields='ohlc')
data = file[symbol]['trend50'][:]
u = pd.DataFrame(data=data[:,3], index=data[:,0].astype('M8[s]'), columns=['up'])#quote.TREND(direction='up', period=50, threshold=0.7)

In [29]:
highchart([
    {'data':quote, 'type':'candlestick'},
    {'data':u, 'type':'line','yaxis':'22', 'name':'uptrend'}
   # {'data':s, 'type':'line','yaxis':'22'},
   # {'data':d, 'type':'line','yaxis':'22'}
])